## Connect to existing Dask cluster

In [1]:
from dask.distributed import LocalCluster

cluster = LocalCluster(n_workers = 70, memory_limit = 'auto', processes = True)

/home/eliu/.virtualenvs/scomp/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35081 instead
  warnings.warn(


In [2]:
cluster.scheduler_address

'tcp://127.0.0.1:40501'

In [3]:
from dask.distributed import LocalCluster, Client

address = 'tcp://127.0.0.1:40501'
client = Client(address)

client

<Client: 'tcp://127.0.0.1:40501' processes=70 threads=140, memory=109.85 GiB>

## Read data set from url

In [4]:
import os              
import urllib 

import dask.dataframe as dd

In [5]:
# data is ~ 150 GB
url = 'https://arcticdata.io/metacat/d1/mn/v2/object/urn%3Auuid%3A27e4043d-75eb-4c4f-9427-0d442526c154'

msg = urllib.request.urlretrieve(url, "dg_soil_moisture.csv")

In [7]:
fp = os.path.join(os.getcwd(),'dg_soil_moisture.csv')

# this only set things up, but nothing is read in yet
df = dd.read_csv(fp, blocksize = '20MB', encoding = 'ISO-8859-1')
df

,timestamp,year,doy,hour,minute,site,logger,port,sensor,sensorZ,m_soil,unit
npartitions=6,,,,,,,,,,,,
,string,int64,int64,int64,int64,string,string,string,string,int64,float64,string
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
# only read in actual data when explicitly specified
df.head(3)

,timestamp,year,doy,hour,minute,site,logger,port,sensor,sensorZ,m_soil,unit
0,2014-07-07 16:30:00,2014,188,16,30,MDF1,MDF1met,Port 3,5TM Moisture/Temp,-6,0.273,m³/m³ VWC
1,2014-07-07 16:30:00,2014,188,16,30,MDF1,MDF1met,Port 4,5TM Moisture/Temp,-11,0.345,m³/m³ VWC
2,2014-07-07 17:00:00,2014,188,17,0,LDF2,LDF2met,Port 3,5TM Moisture/Temp,-8,0.308,m³/m³ VWC


## calculate mean by groupBy

In [14]:
# this does not compute mean, only setting up the computation
averages = df.groupby('year').mean(numeric_only = True)
averages
# this really start computing
averages.compute()

,doy,hour,minute,sensorZ,m_soil
year,,,,,
2014,276.852636,11.513992,15.001123,-11.998332,0.273744
2015,186.720383,11.500723,15.000000,-13.047899,0.263738
2016,183.497453,11.499777,14.999858,-15.000009,0.293595
2017,181.414843,11.499381,15.000144,-14.999981,0.266121
2018,201.824077,11.500796,15.000356,-15.435365,0.282395
2019,173.693311,11.498577,15.000000,-15.124516,0.222193
2020,138.806679,11.489825,14.999322,-15.200054,0.252467


In [17]:
import numpy as np
import dask.array as da

In [19]:
data.shape

(200, 500)

In [18]:
data = np.arange(100_000).reshape(200,500) # _ is just to distinguish how many zeros
a = da.from_array(data, chunks = (100,100)) # chunk size is 100x100
a

dask.array<array, shape=(200, 500), dtype=int64, chunksize=(100, 100), chunktype=numpy.ndarray>

In [20]:
a.mean().compute()

49999.5

## Read in actual ADC data as tif

In [22]:
# download red band
url = 'https://arcticdata.io/metacat/d1/mn/v2/object/urn%3Auuid%3Aac25a399-b174-41c1-b6d3-09974b161e5a'
msg = urllib.request.urlretrieve(url, "RU_ANS_TR2_FL005M_red.tif")


# download nir band
url = 'https://arcticdata.io/metacat/d1/mn/v2/object/urn%3Auuid%3A1762205e-c505-450d-90ed-d4f3e4c302a7'

msg = urllib.request.urlretrieve(url, "RU_ANS_TR2_FL005M_nir.tif")

In [23]:
import rioxarray as rioxr

In [ ]:
# read in the file
fp_red = os.path.join(os.getcwd(),"RU_ANS_TR2_FL005M_red.tif")
red = rioxr.open_rasterio(fp_red, chunks = '15MB')

## 